In [22]:
# Import Libraries

import numpy as np
import pandas as pd
import warnings 
warnings.filterwarnings('ignore')


# Import dataset

df = pd.read_excel('Book2.xlsx')
df.head()

,STT,Mã học phần,Tên học phần,Mã lớp tín chỉ,Số tín chỉ,Số tiết,Số tiết/tuần,Giai đoạn,Tiến độ học\ngiáo dục quốc phòng,Tiến độ,...,Tuần 20,Tuần 21,Tuần 22,Tuần 23,Tuần 24,Tuần 25,Tuần 242,Tuần 253,Column4,Column5
0,146,TMA402,Bảo hiểm trong kinh doanh,TMA402(GD1-HK2-2223).1,3,45,6,1,NaN,09.01-15.01+\n06.02-26.03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,146,TMA402,Bảo hiểm trong kinh doanh,TMA402(GD1-HK2-2223).1,3,45,6,1,NaN,09.01-15.01+\n06.02-26.03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,267,TANE312,BEC Higher,TANE312(GD1-HK2-2223).1,3,54,6,1,NaN,09.01-15.01+\n06.02-02.04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,267,TANE312,BEC Higher,TANE312(GD1-HK2-2223).1,3,54,6,1,NaN,09.01-15.01+\n06.02-02.04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,267,TANE312,BEC Higher,TANE312(GD2-HK2-2223).2,3,54,6,2,NaN,17.04-18.06,...,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
df[df['Mã học phần']=='PLU419']

,STT,Mã học phần,Tên học phần,Mã lớp tín chỉ,Số tín chỉ,Số tiết,Số tiết/tuần,Giai đoạn,Tiến độ học\ngiáo dục quốc phòng,Tiến độ,...,Tuần 20,Tuần 21,Tuần 22,Tuần 23,Tuần 24,Tuần 25,Tuần 242,Tuần 253,Column4,Column5
1118,148,PLU419,Pháp luật trong hoạt động KTĐN,PLU419(GD2-HK2-2223).1,3,45,6,2,NaN,17.04-11.06,...,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1119,148,PLU419,Pháp luật trong hoạt động KTĐN,PLU419(GD2-HK2-2223).1,3,45,6,2,NaN,17.04-11.06,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1120,463,PLU419,Pháp luật trong hoạt động KTĐN,PLU419(GD2-HK2-2223).2,3,45,6,2,NaN,09.01-15.01+\n06.02-26.03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1121,463,PLU419,Pháp luật trong hoạt động KTĐN,PLU419(GD2-HK2-2223).2,3,45,6,2,NaN,09.01-15.01+\n06.02-26.03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
df.columns

Index(['STT', 'Mã học phần', 'Tên học phần', 'Mã lớp tín chỉ', 'Số tín chỉ',
       'Số tiết', 'Số tiết/tuần', 'Giai đoạn',
       'Tiến độ học\ngiáo dục quốc phòng', 'Tiến độ', 'Phân lịch',
       'Tên tổ hợp', 'Thứ', 'Tiết bắt đầu', 'SSCPDK', 'LỊCH HỌC', 'PHÒNG HỌC',
       'Tổng hợp số lớp cần mở'', 'Khóa', 'Chương trình đào tạo',
       'Khóa/ngành học', 'Ghi chú', 'Người thực hiện ĐK', 'Phân loại lớp',
       'Mã lớp', 'Ký kiệu', 'Tuần 1', 'Tuần 2', 'Tuần 3', 'Tuần 4', 'Tuần 5',
       'Tuần 6', 'Tuần 7', 'Tuần 8', 'Tuần 9', 'Tuần 10', 'Tuần 11', 'Tuần 12',
       'Tuần 13', 'Tuần 14', 'Tuần 15', 'Tuần 16', 'Tuần 17', 'Tuần 18',
       'Tuần 19', 'Tuần 20', 'Tuần 21', 'Tuần 22', 'Tuần 23', 'Tuần 24',
       'Tuần 25', 'Tuần 242', 'Tuần 253', 'Column4', 'Column5'],
      dtype='object')

In [25]:
# Clean the data
    ## Choose neccessary columns : 'Mã học phần','Tên học phần', 'Mã lớp tín chỉ', 'Số tín chỉ','Giai đoạn', 'Thứ', 'Tiết bắt đầu', 'LỊCH HỌC', 'Mã lớp'
    ##

In [26]:
cols = ['Mã học phần','Tên học phần', 'Mã lớp tín chỉ', 'Số tín chỉ','Giai đoạn', 'Thứ', 'Tiết bắt đầu', 'LỊCH HỌC', 'Mã lớp']
df_clean = df[cols]

In [27]:
xls = pd.ExcelFile('Timetable Project.xlsx')
df_main= pd.read_excel(xls, 'main')
df_main.head()

,Ca
0,2Ca1.1
1,3Ca1.1
2,3Ca1.1
3,5Ca1.1
4,6Ca1.1


In [28]:
df_clean.rename(columns={'Mã học phần': 'course_code', 'Tên học phần': 'course_name', 'Mã lớp tín chỉ': 'class_code',
                   'Số tín chỉ': 'num_of_credits', 'Giai đoạn': 'period', 'Thứ': 'weekday', 'Tiết bắt đầu': 'start',
                   'LỊCH HỌC': 'time', 'Mã lớp': 'code'}, inplace=True)

In [29]:
df_clean = df_clean[~df_clean['time'].isnull()]

In [30]:
df_clean['time'][
    df_clean['weekday'].isnull()
].unique()

array(['Thứ 2(01-03)', 'Thứ 2(04-06)', 'Thứ 2(10-12)', 'Thứ 3(01-03)',
       'Thứ 2(07-09)'], dtype=object)

In [31]:
list_blank = ['Thứ 2(01-03)', 'Thứ 2(04-06)', 'Thứ 2(10-12)', 'Thứ 3(01-03)','Thứ 2(07-09)']
df_clean['weekday'][df_clean['weekday'].isnull()] = df_clean['time'].map(lambda x: '2Ca1' if x == list_blank[0] else (
                                                '2Ca2' if x == list_blank[1] else(
                                                '2Ca4' if x == list_blank[2] else(
                                                '2Ca4' if x == list_blank[4] else '3Ca1'
                                                )
                                                )
                                                )
                                            )

In [32]:
df_clean['period_weekday'] =  df_clean['weekday'] + '.' + df_clean['period'].astype(str)

In [33]:
df_clean = df_clean[df_clean['period'].isin([1,2])]
df_clean['short_code'] = df_clean['course_code'] + '.' + df_clean['code'].astype(str) + '.' + df_clean['period'].astype(str)

In [41]:
## Giai doan 1 only

df_clean = df_clean[df_clean['period']==1]

In [42]:
set1 = df_clean['short_code'][df_clean['course_code']=='MKT407'].unique().tolist()
set2 = df_clean['short_code'][df_clean['course_code']=='KTE312'].unique().tolist()
set3 = df_clean['short_code'][df_clean['course_code']=='TMA404'].unique().tolist()
set4 = df_clean['short_code'][df_clean['course_code']=='KTE319'].unique().tolist()
set5 = df_clean['short_code'][df_clean['course_code']=='TMA311'].unique().tolist()

all_list = []

import itertools
list_set = [set1, set2, set3, set4, set5]
for element in itertools.product(*list_set):
  all_list.append(element)

[('MKT407.2.1', 'KTE312.1.1', 'TMA404.1.1', 'KTE319.1.1', 'TMA311.2.1'),
 ('MKT407.2.1', 'KTE312.1.1', 'TMA404.1.1', 'KTE319.1.1', 'TMA311.4.1'),
 ('MKT407.2.1', 'KTE312.1.1', 'TMA404.1.1', 'KTE319.1.1', 'TMA311.6.1'),
 ('MKT407.2.1', 'KTE312.1.1', 'TMA404.3.1', 'KTE319.1.1', 'TMA311.2.1'),
 ('MKT407.2.1', 'KTE312.1.1', 'TMA404.3.1', 'KTE319.1.1', 'TMA311.4.1'),
 ('MKT407.2.1', 'KTE312.1.1', 'TMA404.3.1', 'KTE319.1.1', 'TMA311.6.1'),
 ('MKT407.2.1', 'KTE312.1.1', 'TMA404.4.1', 'KTE319.1.1', 'TMA311.2.1'),
 ('MKT407.2.1', 'KTE312.1.1', 'TMA404.4.1', 'KTE319.1.1', 'TMA311.4.1'),
 ('MKT407.2.1', 'KTE312.1.1', 'TMA404.4.1', 'KTE319.1.1', 'TMA311.6.1'),
 ('MKT407.2.1', 'KTE312.3.1', 'TMA404.1.1', 'KTE319.1.1', 'TMA311.2.1'),
 ('MKT407.2.1', 'KTE312.3.1', 'TMA404.1.1', 'KTE319.1.1', 'TMA311.4.1'),
 ('MKT407.2.1', 'KTE312.3.1', 'TMA404.1.1', 'KTE319.1.1', 'TMA311.6.1'),
 ('MKT407.2.1', 'KTE312.3.1', 'TMA404.3.1', 'KTE319.1.1', 'TMA311.2.1'),
 ('MKT407.2.1', 'KTE312.3.1', 'TMA404.3.1', 'KTE319

In [43]:
full_list=[]
for list in all_list:
      xls = pd.ExcelFile('Timetable Project.xlsx')
      df_main= pd.read_excel(xls, 'main')
      for x in list:
            
            list_slots = df_clean['period_weekday'][df_clean['short_code']==x].unique().tolist()
            df_main[x] = 0
            df_main[x] = df_main['Ca'].map(lambda x: 1 if x in list_slots else 0)

      df_main['Sum'] = df_main.sum(axis=1)
      
      if (df_main['Sum']<=1).all():
            full_list.append(list)
      else:
            pass

In [44]:
for list in full_list:
    xls = pd.ExcelFile('Timetable Project.xlsx')
    df_output= pd.read_excel(xls, 'output')
    for x in list:

        ca_list = df_clean['period_weekday'][df_clean['short_code'] == x]
        for ca in ca_list:
            day = int(ca[0])
            slot = ca[1:]
            df_output[day][df_output['Ca']==slot] = x

    print(df_output)
    print('==============================================================================')

      Ca           2           3           4           5           6
0  Ca1.1  TMA404.1.1  MKT407.2.1         NaN  TMA311.4.1         NaN
1  Ca2.1  TMA311.4.1         NaN  TMA404.1.1  MKT407.2.1         NaN
2  Ca3.1         NaN  KTE312.1.1         NaN         NaN  KTE319.1.1
3  Ca4.1         NaN  KTE319.1.1         NaN  KTE312.1.1         NaN
4    sep         sep         sep         sep         sep         sep
5  Ca1.2         NaN         NaN         NaN         NaN         NaN
6  Ca2.2         NaN         NaN         NaN         NaN         NaN
7  Ca3.2         NaN         NaN         NaN         NaN         NaN
8  Ca4.2         NaN         NaN         NaN         NaN         NaN
      Ca           2           3           4           5           6
0  Ca1.1  TMA404.1.1  MKT407.2.1         NaN         NaN         NaN
1  Ca2.1         NaN         NaN  TMA404.1.1  MKT407.2.1         NaN
2  Ca3.1         NaN  KTE312.1.1  TMA311.6.1         NaN  KTE319.1.1
3  Ca4.1         NaN  KTE319.1.1  